# Introduction

In [1]:
import numpy as np
import tensorflow as tf

<div></div>
<img src="../../assets/01_ML_Concepts/LR_GD_1.png", width=500, height=100>

<div>
<img src="../../assets/01_ML_Concepts/LR_GD_2.png", width=500, height=100>
    </div>

<div>
<img src="../../assets/01_ML_Concepts/LR_GD_3.png", width=500, height=100>
    </div>

# Implementation

Let's implement Gradient Descent using the derived gradients

## Synthetic Data

In [2]:
nb_samples = 100 # n in the whiteboard
nb_features = 3 # d in the whiteboard

# generate three random vectors 
x_1 = np.random.normal(0, 0.1, size=nb_samples)
x_2 = np.random.normal(0, 0.3, size=nb_samples)
x_3 = np.random.normal(0, 0.2, size=nb_samples)

# take their linear combination according to vector theta to generate y
noise = np.random.normal(0, 0.01, size=nb_samples) # some gaussian noise
noise = 0
y = 0.3*x_1 + 0.2*x_2 + 0.1*x_3 + noise # number of parameters corresponidng to nb_features
y = y.reshape(-1, 1) # this is important for tf

# create the design matrix X
X = np.stack([x_1, x_2, x_3], axis=1)

# sanity check the dimensions of our matrices
print(f'X shape = {X.shape}, y shape = {y.shape}')

X shape = (100, 3), y shape = (100, 1)


## Numpy

In [3]:
def compute_gradient(y, theta, X):
    '''
    Implement the gradient as shown from the derivation
    '''
    gradient = (-y.T @ X) + (theta.T @ X.T @ X)
    return gradient.T


def compute_loss(y_pred, y_true):
    '''
    Compute the loss function
    '''
    return np.sum(np.square(y_true - y_pred))

In [4]:
# constants and parameters
learning_rate = 0.01
epochs = 500

# initialize theta
theta = np.random.normal(0, 1, size=nb_features).reshape(-1, 1)

for epoch in range(epochs):
    # compute gradients
    gradient = compute_gradient(y, theta, X)
    
    # update variables
    theta -= learning_rate * gradient
    
    # compute current loss
    y_pred = X @ theta
    loss = compute_loss(y_pred, y)
    if epoch % 100 == 0:
        print(f'Epoch = {epoch} loss = {loss:.2f}')
        
print(f'Final Theta: {theta.squeeze()}')

Epoch = 0 loss = 22.69
Epoch = 100 loss = 0.04
Epoch = 200 loss = 0.00
Epoch = 300 loss = 0.00
Epoch = 400 loss = 0.00
Final Theta: [0.29865026 0.19988969 0.09999911]


Nice!

## Tensorflow 2.0


In [5]:
def model(X, theta):
    return tf.matmul(X, theta)


def loss_function(y_pred, y_true):
    return tf.reduce_sum(tf.square(y_pred - y_true))


def train(X, y):
    """
    One Training Epoch
    """
    with tf.GradientTape() as tape:
        # apply the model
        y_pred = tf.matmul(X, theta_tensor)
        
        # compute current loss
        current_loss = tf.reduce_sum(tf.square(y_pred - y))
        
    # automatic differentiation step
    gradients = tape.gradient(current_loss, [theta_tensor])

    # apply gradients
    grads_and_vars = zip(gradients, [theta_tensor])
    optimizer.apply_gradients(grads_and_vars)

    return current_loss

In [6]:
# constants and parameters
learning_rate = 0.01
epochs = 500

# define the data
X_tensor = tf.constant(X, dtype=tf.float32)
y_tensor = tf.constant(y.reshape(-1, 1), dtype=tf.float32)

# initialize theta
theta_tensor = tf.Variable(tf.random.normal(shape=(nb_features, 1)), trainable=True, dtype=tf.float32)

# choose an optimizer
optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)

# run
for epoch in range(epochs):
    current_loss = train(X_tensor, y_tensor)
    if epoch % 1000 == 0:
        print(f'Epoch {epoch}, current loss {current_loss:.2f}')
        
print(f'Final Theta: {theta_tensor.numpy().squeeze()}')

Epoch 0, current loss 7.44
Final Theta: [0.29999152 0.19999929 0.09999996]
